In [9]:
import pandas as pd
import numpy as np
import itertools
import random
from math import log, exp
from statistics import median
from scipy.stats._stats import _kendall_dis
from sklearn.cluster import AgglomerativeClustering

## Loading the sushi data

In [10]:
user_data = pd.read_csv("Sushi_User Data_Filtered.csv")
pref_data = pd.read_csv("Sushi_Preference Rank_Filtered.csv")

In [11]:
user_data.head()

,User ID,Gender,Age,Time Needed,Prev_Prefecture,Prev_Region,Prev_Half,Curr_Prefecture,Curr_Region,Curr_Half,Moved
0,9130,1,3,298,0,0,0,0,0,0,0
1,4849,0,3,303,0,0,0,0,0,0,0
2,3601,0,2,192,0,0,0,0,0,0,0
3,1422,0,3,206,0,0,0,0,0,0,0
4,6767,0,2,447,24,5,0,24,5,0,0


In [12]:
pref_data.head()

,User ID,Rank 1,Rank 2,Rank 3,Rank 4,Rank 5,Rank 6,Rank 7,Rank 8,Rank 9,Rank 10
0,9130,2,7,5,8,0,9,4,3,1,6
1,4849,8,2,7,1,3,0,9,5,6,4
2,3601,7,2,5,8,6,0,3,9,4,1
3,1422,4,2,7,8,3,0,9,6,5,1
4,6767,5,0,3,7,2,1,8,6,9,4


In [13]:
# Saving Users' Region, Full Ranking and Partial Ranking

n = 20
region = user_data.loc[0:n-1,'Curr_Half'].values
or_region = user_data.loc[0:n-1,'Prev_Half'].values
fullranking = pref_data.iloc[0:n,1:11].values
topkranking = pref_data.iloc[0:n,1:5].values

In [14]:
print(region.shape,
      '\n',or_region.shape,
      '\n',fullranking.shape,
      '\n', topkranking.shape)

region

(20,) 
 (20,) 
 (20, 10) 
 (20, 4)


array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1])

## Calculating Kendall Distance between full rankings

In [15]:
# Function to calculate Kendall Distance between 2 permutations

def kendallTau(X, Y):
    
    A = list(X)
    B = list(Y)
    c_d = [0,0]
    
    # Set of all possible object pairs
    pairs = itertools.combinations(range(10), 2)
    
    for x, y in pairs:
        a = A.index(x) - A.index(y) # relative position in Permutation A
        b = B.index(x) - B.index(y) # relative position in Permutation B
        
        if a * b < 0:
            c_d[1] += 1             # if the pair is discordant
        if a * b > 0:
            c_d[0] += 1             # if the pair is concordant
            
    return c_d

In [16]:
# Kendall Distance matrix for full rankings

D = np.zeros( (n, n), dtype=int )
C = np.zeros( (n, n), dtype=int )

for i in range(n):
    
    for j in range(n):
      
        C[i][j],D[i][j] = kendallTau(fullranking[i],fullranking[j])

## Calculating Kernel Bandwidth using median distance criterion

##### Formula from https://arxiv.org/pdf/1708.05106.pdf

In [111]:
# Calculating median distance

l = []

for i in range(n):
    
    for j in range(n):
    
        if i<j:
           
            l.append(D[i][j])

median_dist = median(l)

print('Median distance = ', median_dist)

# Tolerance of sqrt(2) * 10^(-6) recommended by above research paper
delta_square = 2 * pow(10,-12)        

# Denominator of bandwidth formula
Dr = pow( log( (n-1) / delta_square ) , 0.5)

# Kernel bandwidth formula according to above research paper
s = median_dist/Dr

# Scaling factor
v = pow(1/s , 2)

print("Scaling factor for Mallows Kernel: ", v)

Median distance =  18.0
Scaling factor for Mallows Kernel:  0.0922293608473304


## Calculating Tree Purity for full ranking Gram Matrix

In [112]:
G = np.zeros( (n, n), dtype=float )

for i in range(n):
    
    for j in range(n):
        
        # Mallows Kernel using Kendall Distance computed above
        G[i][j] = pow((1 + 1 - 2*exp( (-v) * D[i][j] )), 1)

agg = AgglomerativeClustering(n_clusters=2, affinity='precomputed', linkage='average')
pred_labels = agg.fit_predict(G)

# Tree Purity from Confusion Matrix

CM = np.zeros( (2, 2), dtype=int )

for i in range(n):
    CM[pred_labels[i]][region[i]] += 1
    
purity = 0

for j in range(2):
    purity += CM[j].max()
    
purity /= n

print("\nTree purity for full ranking: ", purity)


Tree purity for full ranking:  0.55


## Generating Monte-Carlo samples consistent with a given partial ranking

### Vanilla set of Monte-Carlo samples

In [82]:
# Assuming all full permutations are equally distributed within each partial ranking set

def MC_vanilla(partial, m):
    
    top = list(partial)
    
    # Listing objects missing from partial ranking
    remaining = list( {0,1,2,3,4,5,6,7,8,9} - set(top) )
    
    # Defining set of m Monte Carlo samples 
    MC_samples = np.zeros( (m, 10), dtype=int )
    
    for i in range(m):
        
        # Each iteration shuffles the "remaining" objects and generates a full ranking consistent with partial
        
        random.shuffle(remaining)
        full = top + remaining        
        MC_samples[i] = np.array(full)

    return MC_samples

### Monte-Carlo set with antithetic pairs

In [83]:
# Function to general set of antithetic pairs of Monte-Carlo samples

def MC_antithetic(partial, m):
    
    top = list(partial)
    
    # Listing objects missing from partial ranking
    remaining = list( {0,1,2,3,4,5,6,7,8,9} - set(top) )
    
    # Defining set of m Monte Carlo samples 
    MC_samples = np.zeros( (2*m, 10), dtype=int )
    
    for i in range(m):
        
        # Each iteration shuffles the "remaining" objects and generates a pair of full rankings
        # full1 is ranking consistent with partial
        # full2 is the antithetic pair of full1 (i.e. where ranking of "remaining" objects is flipped)
        
        random.shuffle(remaining)
        full1 = top + remaining
        remaining.reverse()
        full2 = top + remaining
        
        MC_samples[2*i] = np.array(full1)
        MC_samples[2*i +1] = np.array(full2)

    return MC_samples

### Generating matrices of Monte-Carlo samples for each partial ranking: vanilla and antithetic methods

In [84]:
vanilla_sample_matrix = np.zeros( (n, 20, 10), dtype=int )
antithetic_sample_matrix = np.zeros( (n, 40, 10), dtype=int )

for i in range(n):
    
    vanilla_sample_matrix[i] = MC_vanilla(topkranking[i], 20)
    antithetic_sample_matrix[i] = MC_antithetic(topkranking[i],20)

## Constructing the Monte-Carlo Estimator

In [104]:
# Assuming equal weights to the samples within each set

def MC_estimator(samplesA, samplesB):
    
    N = len(samplesA)
    M = len(samplesB)
    
    v = 0.0922293608473304
    estimate = 0
    
    for i in range(N):
        for j in range(M):
            
            _ , d = kendallTau(samplesA[i],samplesB[j])
            
            estimate += exp( (-v) * d)

    estimate /= N*M
    
    return estimate

## Applying the Estimator

### Vanilla method of generating samples

In [105]:
# Vanilla Estimator

H = np.zeros( (n, n), dtype=float )

for i in range(n):
    for j in range(n):
        
        samplesA = vanilla_sample_matrix[i]
        samplesB = vanilla_sample_matrix[j]
        
        A_A = MC_estimator(samplesA, samplesA)
        B_B = MC_estimator(samplesB, samplesB)
        A_B = MC_estimator(samplesA, samplesB)
        
        H[i][j] = pow( (A_A + B_B - (2*A_B)) , 0.5 )

In [108]:
pred_labels_vanilla = agg.fit_predict(H)

# Tree Purity from Confusion Matrix

CM_vanilla = np.zeros( (2, 2), dtype=int )

for i in range(n):
    CM_vanilla[pred_labels_vanilla[i]][region[i]] += 1
    
purity_vanilla = 0

for j in range(2):
    purity_vanilla += CM_vanilla[j].max()
    
purity_vanilla /= n

print("\nTree purity for Vanilla estimator: ", purity_vanilla)


Tree purity for Vanilla estimator:  0.5


### Antithetic method of generating samples

In [109]:
# Antithetic Estimator

I = np.zeros( (n, n), dtype=float )

for i in range(n):
    for j in range(n):
        
        samplesA = antithetic_sample_matrix[i]
        samplesB = antithetic_sample_matrix[j]
        
        A_A = MC_estimator(samplesA, samplesA)
        B_B = MC_estimator(samplesB, samplesB)
        A_B = MC_estimator(samplesA, samplesB)
        
        I[i][j] = pow( (A_A + B_B - (2*A_B)) , 0.5 )

In [110]:
pred_labels_antithetic = agg.fit_predict(I)

# Tree Purity from Confusion Matrix

CM_antithetic = np.zeros( (2, 2), dtype=int )

for i in range(n):
    CM_antithetic[pred_labels_antithetic[i]][region[i]] += 1
    
purity_antithetic = 0

for j in range(2):
    purity_antithetic += CM_antithetic[j].max()
    
purity_antithetic /= n

print("\nTree purity for Antithetic estimator: ", purity_antithetic)


Tree purity for Antithetic estimator:  0.5
